# NanoPyx-SRRF
A python reimplementation of the original [NanoJ-SRRF](https://github.com/HenriquesLab/NanoJ-SRRF) algorithm

In [ ]:
# Load the dataset to analyse

import os

import ipywidgets as widgets
import skimage
from IPython.display import display

from nanopyx.core.utils.easy_gui import EasyGui
from nanopyx.core.utils.find_files import find_files
from nanopyx.data.download import ExampleDataManager

cwd = os.getcwd()
image_folder = "datasets"
image_files = []
EDM = ExampleDataManager()
example_datasets = EDM.list_datasets()

_path = os.path.join("..", image_folder)
if os.path.exists(_path):
    image_files += find_files(_path, ".tif")
if os.path.exists(image_folder):
    image_files += find_files(image_folder, ".tif")
image_files += ["Example dataset: "+dataset for dataset in example_datasets]

# Create a GUI
gui = EasyGui("srrf")

def on_button_load_data_clicked(b):
    global dataset_original
    if gui["data_source"].value.startswith("Example dataset: "):
        dataset_name = gui["data_source"].value.replace(
            "Example dataset: ", "")
        dataset_original = EDM.get_ZipTiffIterator(dataset_name, as_ndarray=True)
    else:
        dataset_original = skimage.io.imread(gui["data_source"].value)
    gui.save_settings()

gui.add_label("Select data to use:")
gui.add_dropdown("data_source", options=image_files,
                 value="Example dataset: "+example_datasets[4], remember_value=True)
gui.add_button("load_data", description="Load data")
gui["load_data"].on_click(on_button_load_data_clicked)
gui.show()

In [ ]:
import stackview

cropper = stackview.crop(dataset_original, continuous_update=True,
                         zoom_factor=2, slider_text="frame",
                         axis_names=["frame", "y", "x"])
cropper


In [ ]:
dataset = cropper.crop()
dataset

In [ ]:
gui = EasyGui("srrf")
from nanopyx.methods.srrf import SRRF

def run_srrf(b):
    gui.save_settings()
    ring_radius = gui["ring_radius"].value
    magnification = gui["magnification"].value
    do_drift_correction = gui["do_drift_correction"].value
    frames_per_timepoint = gui["frames_per_timepoint"].value

    srrf = SRRF(magnification, ring_radius)
    global dataset_srrf
    dataset_srrf = srrf.calculate(dataset, frames_per_timepoint=frames_per_timepoint)

gui.add_float_slider("ring_radius", description="Ring Radius:", min=0.1, max=3.0, value=0.5, remember_value=True)
gui.add_int_slider("magnification", description="Magnification:", min=1, max=10, value=4)
gui.add_checkbox("do_drift_correction", description="Do drift correction", value=False)
gui.add_label("-=-= Time-Lapse =-=-")
gui.add_int_text("frames_per_timepoint", description="Frames per time-point (0 - auto)", value=0)
gui.add_button("run", description="Run")
gui['run'].on_click(run_srrf)
gui.show()

In [ ]:
import stackview
stackview.curtain(dataset_srrf[0], dataset_srrf[1], continuous_update=True, zoom_factor=1,
                  zoom_spline_order=0)  # , display_width=512, display_height=512)